This notebook will guide you through calculating transport provision of a chosen area, which is fractal dimension of its transport network divided by 2. We will use box counting method to calculate fractal dimension and OSM road network as a transport network. This notebook is inspired by Svitlana Kuznichenko and Adam Tóth.

In [1]:
import arcpy
import scipy
from scipy import optimize
import numpy
from numpy import *
arcpy.env.overwriteOutput = True

print("packages imported")

packages imported


After importing necessary packages, we are going to set paths to our data. Roads and area layers will be stored in a dictionary structure named "inputdata". It has 3 keys: "names", "paths" and "spatial_references", which is now an empty list. The previos two keys have their values in lists as well.

In [42]:
inputdata = {"names": ["roads", "area"],
             "paths": ["D:/diplomka/diplomka/Default.gdb/otm_roads_h123_olomouc_region", "D:/diplomka/diplomka/Default.gdb/NUTS3_olomoucky_kraj"],
             "spatial_references": []}
size = "50 SquareKilometers"
workspace = "D:/diplomka/diplomka/Default.gdb"
cor_sys_string = 'PROJCS["ETRS_1989_LAEA",GEOGCS["GCS_ETRS_1989",DATUM["D_ETRS_1989",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Azimuthal_Equal_Area"],PARAMETER["False_Easting",4321000.0],PARAMETER["False_Northing",3210000.0],PARAMETER["Central_Meridian",10.0],PARAMETER["Latitude_Of_Origin",52.0],UNIT["Meter",1.0]]'
arcpy.env.workspace = workspace

print("data paths and workspace set")

data paths and workspace set


NUTS3 regions were downloaded from this webpage: https://ec.europa.eu/eurostat/web/gisco/geodata/reference-data/administrative-units-statistical-units/nuts

The next step will be uniting coordinate systems. User chose one coordinate sytem and stored it in the cor_sys_string variable. We need to check if our roads and area of interest layers are in that chosen coordinate system. If they're not, they will be projected into it.

In [37]:
cor_sys = arcpy.SpatialReference()
cor_sys.loadFromString(cor_sys_string)
arcpy.env.outputCoordinateSystem = cor_sys
print("chosen coordinate system loaded")

j = 0
for i in inputdata["paths"]:
    inputdata["spatial_references"].append(arcpy.Describe(i).spatialReference)
    if inputdata["spatial_references"][j].factoryCode != cor_sys.factoryCode:
        arcpy.management.Project(i, "reprj_" + inputdata["names"][j], cor_sys)
        inputdata["paths"][j] = workspace + "/" + "reprj_" + inputdata["names"][j]
        print(inputdata["names"][j] + " layer was projected into the chosen coordinate system")
    else:
        print(inputdata["names"][j] + " layer is in the chosen coordinate system")
    j += 1

print(inputdata)

chosen coordinate system loaded
roads layer was projected into the chosen coordinate system
area layer is in the chosen coordinate system
{'names': ['roads', 'area'], 'paths': ['D:/diplomka/diplomka/Default.gdb/reprj_roads', 'D:/diplomka/diplomka/Default.gdb/NUTS3_olomoucky_kraj'], 'spatial_references': [<geoprocessing spatial reference object object at 0x000002257B99CE90>, <geoprocessing spatial reference object object at 0x000002257CBD1250>]}


To make sure that all the roads are within the area of interest, roads layer is clipped by area layer.

In [44]:
arcpy.analysis.Clip(inputdata["paths"][0], inputdata["paths"][1], "clipped_roads")
print("roads clipped by area")

roads clipped by area


Now the hexagonal grid with a cell of the chosen size is generated and clipped by area layer. Transport provision will be calculated for each hexagon. 

In [43]:
arcpy.management.GenerateTessellation("hex_grid", inputdata["paths"][1], "HEXAGON", size)
arcpy.analysis.Clip("hex_grid", inputdata["paths"][1], "hex_gr")
print("hexagonal grid generated and clipped by area layer")

hexagonal grid generated and clipped by area layer


The next step is to cut the roads by hexagons. A layer of roads intersected by hexagons is created where each feature is assigned an FID (Feature ID) of hexagon where it belongs. Now this layer can be dissolved, so every road feature with the same hexagon FID will be dissolved/merged into one feature. The result is a layer with road features where every feature represents road network of their hexagon.

In [45]:
arcpy.analysis.Intersect(["clipped_roads", "hex_gr"], "roads_isect", "ONLY_FID")
arcpy.management.Dissolve("roads_isect", "roads_isect_diss", "FID_hex_gr")
arcpy.management.AddField("hex_gr", "TP", "FLOAT")
print("Roads intersected and dissolved by hexagons and new field added into hex_gr for transport provision")

Roads intersected and dissolved by hexagons and new field added into hex_gr for transport provision


Now the math part begins. 

In [46]:
fitfunc = lambda p, x: (p[0] + p[1] * x)
errfunc = lambda p, x, y: (y - fitfunc(p, x))

a = int(arcpy.management.GetCount("roads_isect_diss").getOutput(0))
rows = arcpy.SearchCursor("roads_isect_diss")
total = []
for row in rows:
    total.append(row.getValue("FID_hex_gr"))
count = int(arcpy.management.GetCount("hex_gr").getOutput(0))
tp_values = []
i = 1

print(f"Number of hexagons which intersect with roads: {a}. Total number of hexagons: {count}")

Number of hexagons which intersect with roads: 138. Total number of hexagons: 151


In [47]:
while i <= a:
    print(f"iteration: {i} out of {a}")
    # select the lines and select the respective hexagon where the lines are
    selected_roads = arcpy.management.SelectLayerByAttribute("roads_isect_diss", "NEW_SELECTION", "OBJECTID = %s" % i)
    selected_hex = arcpy.management.SelectLayerByAttribute("hex_gr", "NEW_SELECTION", "OBJECTID = %s" % (total[i-1]))

    # selected hexagon is exported into layer "one_hex"
    arcpy.conversion.FeatureClassToFeatureClass(selected_hex, workspace, "one_hex")

    # "aa" contains extent of this hexagon
    aa = arcpy.Describe("one_hex").extent
    xmin = aa.XMin
    ymin  = aa.YMin

    # list "squares" contains numbers of squares in fishnet, which will be generated in the next for loop
    squares = [4,16,64,256]
    # list "intersections" contains counts of squares which cover lines in the polygon/hexagon, first it is 1/1, then a/4, b/16, c/64, d/256
    intersections = [1]

    # this for loop has 4 iterations
    # during one iteration it creates fishnet over the "one_hex", clips the fishnet by "one_hex", counts the number of squares that cover lines and add these counts into the list "intersections"
    for n in range(len(squares)):
        arcpy.management.CreateFishnet("fishnet_" + str(squares[n]), str(xmin) + ' ' + str(ymin), str(xmin) + ' ' + str(ymin+1), "0", "0", int((squares[n])**(0.5)), int((squares[n])**(0.5)), "#", "NO_LABELS", "one_hex", "POLYGON")
        arcpy.analysis.Clip("fishnet_" + str(squares[n]), "one_hex", "fishnet_clip_" + str(squares[n]))
        c = int(arcpy.management.GetCount(arcpy.management.SelectLayerByLocation("fishnet_clip_" + str(squares[n]), "INTERSECT", selected_roads)).getOutput(0))
        intersections.append(c)
        arcpy.management.Delete(["fishnet_" + str(squares[n]), "fishnet_clip_" + str(squares[n])])

    # fractal dimension and transport provision calculation (another math which I don't understand, but it works)
    edge = [1,0.5,0.25,0.125,0.0625]
    logx = log(edge)
    logy = log(intersections)
    qout, success = optimize.leastsq(errfunc, [0, 0], args = (logx, logy), maxfev = 30000)

    # adding the transport provision into the list "tp_values" and deleting layer "one_hex"
    # (in the original code, the field TP was calculated in the end of each iteration, in my version, transport provision for each polygon is saved in the list and after the while loop terminates, it is loaded into the field TP)
    tp_values.append(float(int(qout[1]*100000))/(-200000))
    arcpy.management.Delete("one_hex")
    i += 1

iteration: 1 out of 138
iteration: 2 out of 138
iteration: 3 out of 138
iteration: 4 out of 138
iteration: 5 out of 138
iteration: 6 out of 138
iteration: 7 out of 138
iteration: 8 out of 138
iteration: 9 out of 138
iteration: 10 out of 138
iteration: 11 out of 138
iteration: 12 out of 138
iteration: 13 out of 138
iteration: 14 out of 138
iteration: 15 out of 138
iteration: 16 out of 138
iteration: 17 out of 138
iteration: 18 out of 138
iteration: 19 out of 138
iteration: 20 out of 138
iteration: 21 out of 138
iteration: 22 out of 138
iteration: 23 out of 138
iteration: 24 out of 138
iteration: 25 out of 138
iteration: 26 out of 138
iteration: 27 out of 138
iteration: 28 out of 138
iteration: 29 out of 138
iteration: 30 out of 138
iteration: 31 out of 138
iteration: 32 out of 138
iteration: 33 out of 138
iteration: 34 out of 138
iteration: 35 out of 138
iteration: 36 out of 138
iteration: 37 out of 138
iteration: 38 out of 138
iteration: 39 out of 138
iteration: 40 out of 138
iteration

In [48]:
i = 0
with arcpy.da.UpdateCursor("hex_gr", ["OBJECTID", "TP"]) as cursor:
    for row in cursor:
        if row[0] == total[i]:
            row[1] = tp_values[i]
            cursor.updateRow(row)
            if i < a-1:
                i += 1
print("Field TP calculated and updated")

Field TP calculated and updated


In [51]:
arcpy.management.Rename("hex_gr", "fractal_tp")

if arcpy.Exists("reprj_roads"):
    arcpy.management.Delete("reprj_roads")
if arcpy.Exists("reprj_area"):
    arcpy.management.Delete("reprj_area")
if arcpy.Exists("hex_grid"):
    arcpy.management.Delete("hex_grid")
arcpy.management.Delete(["clipped_roads", "roads_isect_diss", "roads_isect"])

del size, workspace, cor_sys, cor_sys_string, i, cursor, row, j, inputdata
del fitfunc, errfunc, a, rows, total, count, tp_values, selected_roads, selected_hex, aa, xmin, ymin, squares, intersections, n, c, edge, logx, logy, qout, success
print("Trash deleted")
print("The script has successfully ended!")


Trash deleted
The script has successfully ended!
